In [5]:
import numpy
import pandas as pd
import os

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Layout
import plotly.io as pio
from plotly.subplots import make_subplots

## Data Preparation

In [1]:
#select model you are interesetd in
#download reps 

## Intrinsic Dimension

In [3]:
#obtain id

In [2]:
#plot id

## Neighboorhod Overlap

In [ ]:
#calculate no

In [ ]:
#plot neigh ov